<a href="https://colab.research.google.com/github/MehrdadDastouri/char_rnn_text_generation/blob/main/char_rnn_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Set random seed for reproducibility
torch.manual_seed(42)

# Sample text data
text = "hello world this is an example of a simple text generation using RNN"
chars = sorted(list(set(text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# Hyperparameters
sequence_length = 10
hidden_size = 128
num_layers = 1
learning_rate = 0.01
epochs = 100

# Prepare the data
def create_dataset(text, sequence_length):
    inputs = []
    targets = []
    for i in range(len(text) - sequence_length):
        input_seq = text[i:i + sequence_length]
        target_seq = text[i + 1:i + sequence_length + 1]
        inputs.append([char_to_idx[char] for char in input_seq])
        targets.append([char_to_idx[char] for char in target_seq])
    return torch.tensor(inputs, dtype=torch.long), torch.tensor(targets, dtype=torch.long)

inputs, targets = create_dataset(text, sequence_length)

# Define the RNN model
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)  # Convert input to embeddings
        out, hidden = self.rnn(x, hidden)  # Pass through RNN
        out = self.fc(out)  # Map to vocabulary size
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

# Initialize the model
vocab_size = len(chars)
model = CharRNN(vocab_size, hidden_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    hidden = model.init_hidden(inputs.size(0))
    optimizer.zero_grad()
    outputs, _ = model(inputs, hidden)
    loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Text generation
def generate_text(model, start_text, length):
    model.eval()
    input_seq = torch.tensor([char_to_idx[char] for char in start_text], dtype=torch.long).unsqueeze(0)
    hidden = model.init_hidden(1)
    generated_text = start_text

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        next_char_idx = torch.argmax(output[:, -1, :]).item()
        next_char = idx_to_char[next_char_idx]
        generated_text += next_char
        input_seq = torch.tensor([[next_char_idx]], dtype=torch.long)

    return generated_text

# Generate and print some text
start_text = "hello"
generated_text = generate_text(model, start_text, length=100)
print(f"Generated Text:\n{generated_text}")

Epoch [10/100], Loss: 0.2945
Epoch [20/100], Loss: 0.1638
Epoch [30/100], Loss: 0.1444
Epoch [40/100], Loss: 0.1412
Epoch [50/100], Loss: 0.1400
Epoch [60/100], Loss: 0.1396
Epoch [70/100], Loss: 0.1394
Epoch [80/100], Loss: 0.1393
Epoch [90/100], Loss: 0.1392
Epoch [100/100], Loss: 0.1392
Generated Text:
hello world this is an example of a simple text generation using RNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
